# SKU Discount Handler Module

Handles creation, activation, and deactivation of SKU discounts.

## Input Requirements
This module receives the **Module 3 output file** and processes SKU discounts.

**Required columns from Module 3:**
- `product_id` - Product ID
- `warehouse_id` - Warehouse ID  
- `activate_sku_discount` - Boolean (True = create/keep discount)
- `sku_discount_value` - Discount percentage (e.g., 5 for 5%)

## Workflow
1. Load Module 3 output file
2. Deactivate ALL existing SKU discounts (clean slate)
3. Create new SKU discounts where `activate_sku_discount = True`

## Output
- Summary of actions taken
- Log file with details


In [ ]:
# =============================================================================
# IMPORTS & CONFIGURATION
# =============================================================================
import pandas as pd
import requests
from datetime import datetime, timedelta
import pytz
import glob
import os

# Cairo Timezone
CAIRO_TZ = pytz.timezone('Africa/Cairo')
CAIRO_NOW = datetime.now(CAIRO_TZ)
TODAY = CAIRO_NOW.date()

# =============================================================================
# API CONFIGURATION (TO BE FILLED WITH ACTUAL ENDPOINTS)
# =============================================================================
API_BASE_URL = "https://your-api-endpoint.com/api"
API_KEY = "your-api-key"
HEADERS = {
    "Authorization": f"Bearer {API_KEY}",
    "Content-Type": "application/json"
}

# Default discount settings
DEFAULT_DISCOUNT_DURATION_HOURS = 12  # Discount valid until next run
DEFAULT_COHORT_ID = None  # Set if using cohort-based discounts

# =============================================================================
# INPUT FILE (Module 3 Output)
# =============================================================================
# Option 1: Specify exact file
# MODULE_3_OUTPUT = 'module_3_output_20260120_1500.xlsx'

# Option 2: Auto-find latest Module 3 output from today
def get_latest_module3_output():
    pattern = f'module_3_output_{TODAY.strftime("%Y%m%d")}_*.xlsx'
    files = glob.glob(pattern)
    if files:
        return max(files, key=os.path.getctime)
    return None

MODULE_3_OUTPUT = get_latest_module3_output()

print(f"SKU Discount Handler")
print(f"{'='*50}")
print(f"Cairo Time: {CAIRO_NOW.strftime('%Y-%m-%d %H:%M')}")
print(f"Input File: {MODULE_3_OUTPUT}")


In [ ]:
# =============================================================================
# LOAD MODULE 3 OUTPUT
# =============================================================================
if MODULE_3_OUTPUT is None:
    raise FileNotFoundError("No Module 3 output file found for today!")

df = pd.read_excel(MODULE_3_OUTPUT)
print(f"Loaded {len(df)} records from Module 3 output")

# Filter to SKUs that need SKU discount activation
df_to_activate = df[
    (df['activate_sku_discount'] == True) & 
    (df['sku_discount_value'] > 0)
].copy()

print(f"\nSKUs to activate SKU discount: {len(df_to_activate)}")
print(f"Unique warehouses: {df_to_activate['warehouse_id'].nunique()}")

# Preview
if len(df_to_activate) > 0:
    print(f"\nPreview of SKUs to get discounts:")
    display(df_to_activate[['product_id', 'warehouse_id', 'sku', 'sku_discount_value', 'action_reason']].head(10))


In [ ]:
# =============================================================================
# API FUNCTIONS (TO BE IMPLEMENTED WITH ACTUAL ENDPOINTS)
# =============================================================================

def deactivate_all_sku_discounts(warehouse_ids: list) -> dict:
    """
    Deactivate ALL active SKU discounts for given warehouses.
    
    Args:
        warehouse_ids: List of warehouse IDs to deactivate discounts for
        
    Returns:
        dict with 'success', 'deactivated_count', 'errors'
    """
    # TODO: Replace with actual API call
    # Example:
    # response = requests.post(
    #     f"{API_BASE_URL}/sku-discounts/bulk-deactivate",
    #     headers=HEADERS,
    #     json={"warehouse_ids": warehouse_ids}
    # )
    # return response.json()
    
    print(f"[STUB] Would deactivate all SKU discounts for warehouses: {warehouse_ids}")
    return {
        "success": True,
        "deactivated_count": 0,
        "errors": []
    }


def create_sku_discount(
    product_id: int,
    warehouse_id: int,
    discount_value: float,
    start_date: datetime = None,
    end_date: datetime = None,
    cohort_id: int = None
) -> dict:
    """
    Create a new SKU discount.
    
    Args:
        product_id: Product ID
        warehouse_id: Warehouse ID
        discount_value: Discount percentage (e.g., 5 for 5%)
        start_date: When discount starts (default: now)
        end_date: When discount ends (default: now + DEFAULT_DISCOUNT_DURATION_HOURS)
        cohort_id: Optional cohort ID for targeted discounts
        
    Returns:
        dict with 'success', 'discount_id', 'message'
    """
    if start_date is None:
        start_date = CAIRO_NOW
    if end_date is None:
        end_date = CAIRO_NOW + timedelta(hours=DEFAULT_DISCOUNT_DURATION_HOURS)
    
    # TODO: Replace with actual API call
    # payload = {
    #     "product_id": product_id,
    #     "warehouse_id": warehouse_id,
    #     "discount_percentage": discount_value,
    #     "start_at": start_date.isoformat(),
    #     "end_at": end_date.isoformat(),
    #     "cohort_id": cohort_id
    # }
    # response = requests.post(
    #     f"{API_BASE_URL}/sku-discounts",
    #     headers=HEADERS,
    #     json=payload
    # )
    # return response.json()
    
    return {
        "success": True,
        "discount_id": None,
        "message": f"Created {discount_value}% discount"
    }


def bulk_create_sku_discounts(discounts_df: pd.DataFrame) -> dict:
    """
    Bulk create SKU discounts from DataFrame.
    
    Args:
        discounts_df: DataFrame with columns:
            - product_id
            - warehouse_id  
            - sku_discount_value (percentage)
            
    Returns:
        dict with 'success', 'created_count', 'failed_count', 'errors'
    """
    # TODO: Replace with actual bulk API call if available
    # For now, iterate and create individually
    
    created = 0
    failed = 0
    errors = []
    
    total = len(discounts_df)
    for idx, row in discounts_df.iterrows():
        result = create_sku_discount(
            product_id=int(row['product_id']),
            warehouse_id=int(row['warehouse_id']),
            discount_value=float(row['sku_discount_value'])
        )
        if result['success']:
            created += 1
        else:
            failed += 1
            errors.append({
                'product_id': row['product_id'],
                'warehouse_id': row['warehouse_id'],
                'error': result.get('message', 'Unknown error')
            })
        
        # Progress update every 100 records
        if (idx + 1) % 100 == 0:
            print(f"  Progress: {idx + 1}/{total} processed...")
    
    return {
        "success": failed == 0,
        "created_count": created,
        "failed_count": failed,
        "errors": errors
    }

print("API functions defined ✓")


In [ ]:
# =============================================================================
# STEP 1: DEACTIVATE ALL EXISTING SKU DISCOUNTS
# =============================================================================
print("STEP 1: Deactivating existing SKU discounts...")
print("="*50)

warehouse_ids = df['warehouse_id'].unique().tolist()
print(f"Warehouses to process: {warehouse_ids}")

deactivate_result = deactivate_all_sku_discounts(warehouse_ids)

print(f"\nDeactivation Result:")
print(f"  Success: {deactivate_result['success']}")
print(f"  Deactivated: {deactivate_result['deactivated_count']}")
if deactivate_result['errors']:
    print(f"  Errors: {len(deactivate_result['errors'])}")


In [ ]:
# =============================================================================
# STEP 2: CREATE NEW SKU DISCOUNTS
# =============================================================================
print("\nSTEP 2: Creating new SKU discounts...")
print("="*50)

if len(df_to_activate) == 0:
    print("No SKU discounts to create.")
    create_result = {"success": True, "created_count": 0, "failed_count": 0, "errors": []}
else:
    print(f"Creating {len(df_to_activate)} SKU discounts...")
    create_result = bulk_create_sku_discounts(df_to_activate)
    
    print(f"\nCreation Result:")
    print(f"  Success: {create_result['success']}")
    print(f"  Created: {create_result['created_count']}")
    print(f"  Failed: {create_result['failed_count']}")
    
    if create_result['errors']:
        print(f"\nErrors ({len(create_result['errors'])}):")
        for err in create_result['errors'][:10]:  # Show first 10
            print(f"  - Product {err['product_id']}, Warehouse {err['warehouse_id']}: {err['error']}")


In [ ]:
# =============================================================================
# SUMMARY
# =============================================================================
print("\n" + "="*60)
print("SKU DISCOUNT HANDLER - SUMMARY")
print("="*60)
print(f"Timestamp: {CAIRO_NOW.strftime('%Y-%m-%d %H:%M')} Cairo Time")
print(f"Input File: {MODULE_3_OUTPUT}")
print(f"\nResults:")
print(f"  Total SKUs in input: {len(df)}")
print(f"  SKUs needing discount: {len(df_to_activate)}")
print(f"  Discounts deactivated: {deactivate_result['deactivated_count']}")
print(f"  Discounts created: {create_result['created_count']}")
print(f"  Discounts failed: {create_result['failed_count']}")
print("="*60)

# Breakdown by discount value
if len(df_to_activate) > 0:
    print("\nDiscount Value Breakdown:")
    print(df_to_activate['sku_discount_value'].value_counts().to_string())
